In [1]:
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.axes_grid1 import make_axes_locatable
import IPython.display as ipd
import numpy as np
%matplotlib notebook
from scipy.signal import lfilter

## Phasor

In [2]:
# analog(continuous) ㄴㄴ, digital(discrete)
# parameter setting
amp = 1          # range [0.0, 1.0]
sr = 10000       # sampling rate, Hz, 가청주파수: 20000Hz --> sr : 40000Hz
dur = 0.5        # in seconds
freq = 100.0     # sine frequency, Hz

In [3]:
# generate time
# 0.0001초~0.5초까지 만든 것
t = np.arange(1, sr * dur+1)/sr
t

array([1.000e-04, 2.000e-04, 3.000e-04, ..., 4.998e-01, 4.999e-01,
       5.000e-01])

In [4]:
t.shape

(5000,)

In [6]:
# generate phase
theta = t * 2*np.pi * freq 
theta

array([6.28318531e-02, 1.25663706e-01, 1.88495559e-01, ...,
       3.14033602e+02, 3.14096434e+02, 3.14159265e+02])